# Neenopal Python Dummy Data Task

In [172]:
import pandas as pd
import mysql.connector

# Question 1

## Read the csv or excel file

In [173]:
df = pd.read_csv('used_bikes.csv') # reading the data using the pandas read_csv function

In [174]:
df.head()

,bike_name,price,city,kms_driven,owner,age,power,brand
0,TVS Star City Plus Dual Tone 110cc,35000.0,Ahmedabad,17654.0,First Owner,3.0,110.0,TVS
1,Royal Enfield Classic 350cc,119900.0,Delhi,11000.0,First Owner,4.0,350.0,Royal Enfield
2,Triumph Daytona 675R,600000.0,Delhi,110.0,First Owner,8.0,675.0,Triumph
3,TVS Apache RTR 180cc,65000.0,Bangalore,16329.0,First Owner,4.0,180.0,TVS
4,Yamaha FZ S V 2.0 150cc-Ltd. Edition,80000.0,Bangalore,10000.0,First Owner,3.0,150.0,Yamaha


### The dataset consists of information about different bikes, including their name, price, city, kilometers driven, owner type, age, power, and brand.

In [175]:
df.dtypes

bike_name      object
price         float64
city           object
kms_driven    float64
owner          object
age           float64
power         float64
brand          object
dtype: object

# Question 2

## Change the datatypes of the column and check the memory usage before and after the change in the data types.

In [176]:
df.memory_usage()

Index          128
bike_name     1192
price         1192
city          1192
kms_driven    1192
owner         1192
age           1192
power         1192
brand         1192
dtype: int64

In [177]:
# Checking the memory usage before the data type changes
before_memory_usage = df.memory_usage().sum()
before_memory_usage

9664

In [178]:
# Changing the data types of the float columns to int32
float_columns = ['price', 'kms_driven', 'age', 'power']
for column in float_columns:
    df[column] = df[column].astype('int32')

In [179]:
df.memory_usage()

Index          128
bike_name     1192
price          596
city          1192
kms_driven     596
owner         1192
age            596
power          596
brand         1192
dtype: int64

In [180]:
# Checking the memory usage before the data type changes
after_memory_usage = df.memory_usage().sum()
after_memory_usage

7280

# Question 3

## Dump the data into the mysql database.


In [181]:
#Removing indexing from the table
df.reset_index(drop=True, inplace=True)
print(df.head())

                              bike_name   price       city  kms_driven  \
0    TVS Star City Plus Dual Tone 110cc   35000  Ahmedabad       17654   
1           Royal Enfield Classic 350cc  119900      Delhi       11000   
2                  Triumph Daytona 675R  600000      Delhi         110   
3                  TVS Apache RTR 180cc   65000  Bangalore       16329   
4  Yamaha FZ S V 2.0 150cc-Ltd. Edition   80000  Bangalore       10000   

         owner  age  power          brand  
0  First Owner    3    110            TVS  
1  First Owner    4    350  Royal Enfield  
2  First Owner    8    675        Triumph  
3  First Owner    4    180            TVS  
4  First Owner    3    150         Yamaha  


In [182]:
db_host = "localhost"
db_user = "root"
db_password = "kvsteja09092003"
db_name = "bikes"
table_name = "bike_info" 

# Connecting to the MySQL database
connection = mysql.connector.connect(host=db_host, user=db_user, password=db_password, database=db_name)
cursor = connection.cursor()

### Deleting existing indexing from the table

In [183]:
 try:
    drop_index_query = f"DROP INDEX idx_{table_name}_id ON {table_name};"
    cursor.execute(drop_index_query)
except mysql.connector.Error as e:
    pass

## Insert data into the table

In [184]:
# Insert data into the table
for index, row in df.iterrows():
    values = tuple(row)
    insert_query = f"INSERT INTO {table_name} (bike_name, price, city, kms_driven, owner, age, power, brand) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"
    cursor.execute(insert_query, values)

In [185]:
# Commit the changes to the database
connection.commit()

# Question 4 

## index the column after the data is inserted

In [186]:
# Index the column
index_query = f"CREATE INDEX idx_{table_name}_id ON {table_name}(id);"
cursor = connection.cursor()
cursor.execute(index_query)

In [187]:
connection.commit()

In [188]:
# Show the indexed data
cursor.execute(f"SELECT * FROM {table_name} ORDER BY id;")
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'TVS Star City Plus Dual Tone 110cc', 35000, 'Ahmedabad', 17654, 'First Owner', 3, 110, 'TVS')
(2, 'Royal Enfield Classic 350cc', 119900, 'Delhi', 11000, 'First Owner', 4, 350, 'Royal Enfield')
(3, 'Triumph Daytona 675R', 600000, 'Delhi', 110, 'First Owner', 8, 675, 'Triumph')
(4, 'TVS Apache RTR 180cc', 65000, 'Bangalore', 16329, 'First Owner', 4, 180, 'TVS')
(5, 'Yamaha FZ S V 2.0 150cc-Ltd. Edition', 80000, 'Bangalore', 10000, 'First Owner', 3, 150, 'Yamaha')
(6, 'Yamaha FZs 150cc', 53499, 'Delhi', 25000, 'First Owner', 6, 150, 'Yamaha')
(7, 'Honda CB Hornet 160R  ABS DLX', 85000, 'Delhi', 8200, 'First Owner', 3, 160, 'Honda')
(8, 'Hero Splendor Plus Self Alloy 100cc', 45000, 'Delhi', 12645, 'First Owner', 3, 100, 'Hero')
(9, 'Royal Enfield Thunderbird X 350cc', 145000, 'Bangalore', 9190, 'First Owner', 3, 350, 'Royal Enfield')
(10, 'Royal Enfield Classic Desert Storm 500cc', 88000, 'Delhi', 19000, 'Second Owner', 7, 500, 'Royal Enfield')
(11, 'Yamaha YZF-R15 2.0 150cc', 72000, 

In [189]:
cursor.close()
connection.close()